In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from plotly.offline import init_notebook_mode, iplot

from wordcloud import WordCloud
from wordcloud import STOPWORDS

import matplotlib.pyplot as plt
import plotly.graph_objs as go
from wordcloud import WordCloud
from textblob import TextBlob 

#import plotly.plotly as py
from plotly import tools
import seaborn as sns
import pandas as pd
import string, os, random

init_notebook_mode(connected=True)
punc = string.punctuation

/kaggle/input/healthcareworkersburnout/df.csv
/kaggle/input/burnout-dataframe/burnout.csv
/kaggle/input/burnout-dataframe/__results__.html
/kaggle/input/burnout-dataframe/df.csv
/kaggle/input/burnout-dataframe/__notebook__.ipynb
/kaggle/input/burnout-dataframe/__output__.json
/kaggle/input/burnout-dataframe/burnout2.csv
/kaggle/input/burnout-dataframe/custom.css
/kaggle/input/burnout-tweets-analysis/__results__.html
/kaggle/input/burnout-tweets-analysis/df.csv
/kaggle/input/burnout-tweets-analysis/__notebook__.ipynb
/kaggle/input/burnout-tweets-analysis/__output__.json
/kaggle/input/burnout-tweets-analysis/custom.css
/kaggle/input/burnout-tweets-analysis/__results___files/__results___61_0.png
/kaggle/input/burnout-tweets-analysis/__results___files/__results___63_0.png
/kaggle/input/burnout-tweets-analysis/__results___files/__results___59_0.png
/kaggle/input/burnout-tweets-analysis/__results___files/__results___51_0.png


In [2]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
df = pd.read_csv('../input/healthcareworkersburnout/df.csv')
df

,Unnamed: 0,Unnamed: 0.1,Tweet Date,Tweet ID,Tweet Text,tweet_source,tweet_retweet_count,tweet_favorite_count,word_count,char_count,...,year,month,day,unique_word_count,stop_word_count,url_count,mean_word_length,hashtag_count,mention_count,burnout
0,0,0,2021-01-19,1351590096752017416,I'm a big music person. It speaks to me in all...,Twitter for Android,0,6,27,142,...,2021,1,19,25,15,1,4.296296,0,0,1
1,1,1,2021-01-19,1351555038863368193,@JohnWHarris15 I adore you and your unlimited ...,Twitter for Android,0,1,9,59,...,2021,1,19,9,4,0,5.555556,0,1,1
2,2,2,2021-01-19,1351554892926750720,@bubblydncer I have replied to countless texts...,Twitter for Android,0,1,22,142,...,2021,1,19,20,10,1,5.500000,0,1,1
3,3,3,2021-01-19,1351554395234824192,@MelBeer93 Ahhh I love this!,Twitter for Android,0,0,5,29,...,2021,1,19,5,1,0,4.800000,0,1,0
4,4,4,2021-01-19,1351554266247426050,@TheKimClub https://t.co/UUx3417HcD,Twitter for Android,0,1,2,35,...,2021,1,19,2,0,1,17.000000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874,1874,2026,2021-01-11,1348687433978798080,in my lecture rn talking about covid,Twitter for iPhone,0,0,7,37,...,2021,1,11,7,3,0,4.285714,0,0,1
1875,1875,2027,2021-01-11,1348687432938647552,@buzzycalz @skinnyqweeen @hearteyezshawty im i...,Twitter for iPhone,0,2,27,174,...,2021,1,11,26,11,0,5.444444,0,3,1
1876,1876,2028,2021-01-11,1348687432913473536,"@Death13__ Also this is just streams, videos a...",Twitter for Android,0,1,16,84,...,2021,1,11,16,8,0,4.312500,0,1,1
1877,1877,2029,2021-01-11,1348687432904892416,@Lyssss69 Honestly I just saw these on tiktok ...,Twitter for iPhone,0,1,15,74,...,2021,1,11,14,8,0,4.000000,0,1,0


In [4]:
df.tail()

,Unnamed: 0,Unnamed: 0.1,Tweet Date,Tweet ID,Tweet Text,tweet_source,tweet_retweet_count,tweet_favorite_count,word_count,char_count,...,year,month,day,unique_word_count,stop_word_count,url_count,mean_word_length,hashtag_count,mention_count,burnout
1874,1874,2026,2021-01-11,1348687433978798080,in my lecture rn talking about covid,Twitter for iPhone,0,0,7,37,...,2021,1,11,7,3,0,4.285714,0,0,1
1875,1875,2027,2021-01-11,1348687432938647552,@buzzycalz @skinnyqweeen @hearteyezshawty im i...,Twitter for iPhone,0,2,27,174,...,2021,1,11,26,11,0,5.444444,0,3,1
1876,1876,2028,2021-01-11,1348687432913473536,"@Death13__ Also this is just streams, videos a...",Twitter for Android,0,1,16,84,...,2021,1,11,16,8,0,4.312500,0,1,1
1877,1877,2029,2021-01-11,1348687432904892416,@Lyssss69 Honestly I just saw these on tiktok ...,Twitter for iPhone,0,1,15,74,...,2021,1,11,14,8,0,4.000000,0,1,0
1878,1878,2030,2021-01-11,1348687428589035522,@ariellelynn__ Seriously my favorite rn,Twitter for iPhone,0,0,5,39,...,2021,1,11,5,1,0,7.000000,0,1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1879 entries, 0 to 1878
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            1879 non-null   int64  
 1   Unnamed: 0.1          1879 non-null   int64  
 2   Tweet Date            1879 non-null   object 
 3   Tweet ID              1879 non-null   int64  
 4   Tweet Text            1879 non-null   object 
 5   tweet_source          1879 non-null   object 
 6   tweet_retweet_count   1879 non-null   int64  
 7   tweet_favorite_count  1879 non-null   int64  
 8   word_count            1879 non-null   int64  
 9   char_count            1879 non-null   int64  
 10  word_density          1879 non-null   float64
 11  punc_count            1879 non-null   int64  
 12  sentiment             1879 non-null   float64
 13  year                  1879 non-null   int64  
 14  month                 1879 non-null   int64  
 15  day                  

In [6]:
df.describe()

,Unnamed: 0,Unnamed: 0.1,Tweet ID,tweet_retweet_count,tweet_favorite_count,word_count,char_count,word_density,punc_count,sentiment,year,month,day,unique_word_count,stop_word_count,url_count,mean_word_length,hashtag_count,mention_count,burnout
count,1879.000000,1879.000000,1.879000e+03,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.00000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000,1879.000000
mean,939.000000,1048.265035,1.327972e+18,4.282597,42.165514,14.844598,89.045769,0.193180,5.631719,0.072220,2020.598723,3.871740,13.43587,14.042576,6.248536,0.376796,5.778453,0.118148,0.698244,0.646621
std,542.564896,591.363305,3.366748e+16,34.185665,212.083711,10.179921,58.309802,0.056461,4.725542,0.314292,0.490287,4.038163,6.09055,9.038157,5.291248,0.493422,2.891427,0.840207,1.454292,0.478147
min,0.000000,0.000000,1.236776e+18,0.000000,0.000000,1.000000,7.000000,0.041667,0.000000,-1.000000,2020.000000,1.000000,1.00000,1.000000,0.000000,0.000000,2.400000,0.000000,0.000000,0.000000
25%,469.500000,538.000000,1.320073e+18,0.000000,0.000000,7.000000,44.000000,0.161290,2.000000,0.000000,2020.000000,1.000000,11.00000,7.000000,2.000000,0.000000,4.300000,0.000000,0.000000,0.000000
50%,939.000000,1087.000000,1.348688e+18,0.000000,2.000000,13.000000,73.000000,0.198020,5.000000,0.000000,2021.000000,1.000000,11.00000,12.000000,5.000000,0.000000,4.958333,0.000000,1.000000,1.000000
75%,1408.500000,1558.500000,1.348689e+18,0.000000,10.000000,22.000000,141.000000,0.227273,9.000000,0.200000,2021.000000,7.000000,16.00000,20.000000,10.000000,1.000000,6.082492,0.000000,1.000000,1.000000
max,1878.000000,2030.000000,1.351590e+18,838.000000,5332.000000,92.000000,912.000000,0.400000,60.000000,1.000000,2021.000000,12.000000,31.00000,89.000000,35.000000,2.000000,23.000000,19.000000,49.000000,1.000000


In [7]:
df.nunique()

Unnamed: 0              1879
Unnamed: 0.1            1879
Tweet Date               189
Tweet ID                1879
Tweet Text              1879
tweet_source              14
tweet_retweet_count       67
tweet_favorite_count     222
word_count                59
char_count               227
word_density             601
punc_count                28
sentiment                361
year                       2
month                     11
day                       31
unique_word_count         51
stop_word_count           31
url_count                  3
mean_word_length         622
hashtag_count             13
mention_count             11
burnout                    2
dtype: int64

# Call to Action

I was listening to a growth and leadership podcast this morning and it talked about burnout. It is seen in leadership - a lot. Often times, it is due to people giving excuses that it is only temporary and so they need to just push through during a season. More often, it is due to not listening to warning signs such as irritability or lack of basic health needs (sleep, food, relationships). If those we need to take care of our health are suffering from burnout, then there needs to be a source to determine that there is a trend towards burnout or that they are already there. 

In this notebook, I will be looking at tweets and building models that will help paint a picture of whether or not someone needs intervention and refueling.

Thank you: https://www.kaggle.com/shivamb/seconds-from-disaster-text-eda-and-analysis

## Bag of Words Model

BoW will be used to tag tweets as burnout or not. The same approach will be used to see what sort of burnout occurred. 

Bag of Words approach is a simple dictionary based approach in which a lookup (dictionary) of known words is used to check weather a given document belongs to a particular class or not. It is more like a Naive Bayes approach as it measures the occurance of known dictionary based words in the given document, and assigns a probability of the target class.

* Words Usage
* Topic Modelling using LDA

In this notebook, I am focusing on burnout causes as a topic and will be using a simple dictionary of 12 known burnout words.

In [8]:
# function to check if a bag's element is present in the text
def get_category(txt, bag):
    category = [x for x in bag if x in txt.lower()]
    if not category:
        category = [""]
    return category[0] 

Burnout bag was based on burnout bigrams list.

In [9]:
burnout_bag = ['cry', 'crying', 'nervous', 'sorry', 'can\'t take this', 'crazy', 'hungry', 'anxious', 'insomnia', 'depress', 'can\'t think', 'can\'t focus', 'angry', 'anger', 'idc', 'sad', 'unhappy', 'pessimistic', 'alone', 'detached', 'apathetic', 'hopeless', 'irritated', 'tired', 'exhausted', 'long hours', 'no satisfaction', 'not satsified', 'hard', 'feel bad', 'bad', 'shaking', 'pain', 'scream', 'confused', 'attack', 'panic', 'scared', 'frightened', 'another one', 'losing mind', 'mad', 'mental health', 'oh god', 'days off', 'feel sick', 'struggling', 'sucks', 'mess', 'sleep', 'bad place', 'speechless']
df['burnout_indicators'] = df['Tweet Text'].apply(lambda x : get_category(x, burnout_bag)) # idc = i don't care (apathy)

In [10]:
df.burnout_indicators.value_counts()

                 1573
sorry              39
hard               36
cry                33
bad                30
mad                30
sleep              18
mess               12
sad                12
tired              10
sucks              10
pain                9
crazy               8
scared              6
alone               5
days off            5
angry               4
confused            4
anxious             3
mental health       3
scream              3
attack              3
speechless          2
nervous             2
depress             2
anger               2
can't think         2
hungry              2
shaking             2
struggling          2
insomnia            2
feel sick           1
feel bad            1
irritated           1
exhausted           1
can't focus         1
Name: burnout_indicators, dtype: int64

## Words Usage: Distribution of Burnout

In [11]:
labels = list(df['burnout_indicators'].value_counts().index)[1:]
values = list(df['burnout_indicators'].value_counts().values)[1:]
colors = ['lightblue','gray','#eee','#999', '#9f9f']
trace = go.Pie(labels=labels, values=values, hoverinfo='label+percent', 
               textinfo='value', name='tweet counts of different burnout indicators',
               marker=dict(colors=colors))
layout = dict(title = 'Distribution of burnout tweets',
              xaxis= dict(title= 'World Rank',ticklen= 5,zeroline= False)
             )
fig = dict(data = [trace], layout = layout)
iplot(fig)

## Topic Modeling using LDA

In [12]:
# stopwords = open(path+"country-names-codes/stopwords.txt").read().strip().split("\n")
# stopwords = [x.replace("\r","") for x in stopwords]
from collections import Counter 

def clean_text(txt):    
    txt = txt.lower()
    txt = "".join(x for x in txt if x not in punc)
    words = txt.split()
    words = [wrd for wrd in words if wrd not in STOPWORDS]
    words = [wrd for wrd in words if len(wrd) > 1]
    txt = " ".join(words)
    return txt

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np 

def generate_topic_models(category):
    small_df = df[df.burnout_indicators==category]
    small_df['clean_text'] = small_df['Tweet Text'].apply(clean_text)

    cvectorizer = CountVectorizer(min_df=4, max_features=4000, ngram_range=(1,2))
    cvz = cvectorizer.fit_transform(small_df['clean_text'])

    lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', max_iter=20, random_state=42)
    X_topics = lda_model.fit_transform(cvz)

    topic_word = lda_model.components_ 
    vocab = cvectorizer.get_feature_names()
    return topic_word, vocab 

### Sorry - Topic Models

In [14]:
n_top_words = 10
topic_word, vocab = generate_topic_models("sorry")
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print ("Topic " + str(i+1) + ": " + " | ".join(topic_words) + "\n")

Topic 1: im | sorry | im sorry | rn | sorry loss | loss

Topic 2: sorry loss | loss | sorry | im | im sorry | rn

Topic 3: im | sorry | rn | im sorry | sorry loss | loss

Topic 4: loss | sorry loss | im | im sorry | rn | sorry

Topic 5: sorry loss | loss | sorry | im | im sorry | rn

Topic 6: sorry | rn | im | im sorry | sorry loss | loss

Topic 7: sorry | rn | sorry loss | im sorry | loss | im

Topic 8: rn | sorry | im | im sorry | loss | sorry loss

Topic 9: im | sorry | im sorry | rn | sorry loss | loss

Topic 10: sorry | im | im sorry | rn | sorry loss | loss



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Hard - Topic Models

In [15]:
n_top_words = 10
topic_word, vocab = generate_topic_models("hard")
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print ("Topic " + str(i+1) + ": " + " | ".join(topic_words) + "\n")

Topic 1: rn | hard | hard rn | im | trying | know | time

Topic 2: hard | time | rn | hard rn | trying | know | im

Topic 3: rn | trying | hard rn | know | hard | im | time

Topic 4: know | rn | time | hard | im | trying | hard rn

Topic 5: hard rn | trying | time | im | know | hard | rn

Topic 6: hard | rn | im | trying | hard rn | time | know

Topic 7: hard | time | know | rn | im | trying | hard rn

Topic 8: know | hard | trying | rn | time | im | hard rn

Topic 9: im | know | trying | hard rn | time | hard | rn

Topic 10: hard | rn | im | hard rn | trying | time | know



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Cry - Topic Models

In [16]:
n_top_words = 10
topic_word, vocab = generate_topic_models("cry")
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print ("Topic " + str(i+1) + ": " + " | ".join(topic_words) + "\n")

Topic 1: crypto | im | cant | cry | crying | crying rn | rn

Topic 2: im | cant | crying | crypto | cry | rn | crying rn

Topic 3: rn | crying | cry | crypto | crying rn | cant | im

Topic 4: crying | im | crying rn | crypto | rn | cant | cry

Topic 5: cry | rn | cant | crying | im | crying rn | crypto

Topic 6: rn | cant | im | crypto | crying | crying rn | cry

Topic 7: crying rn | crying | rn | im | cant | crypto | cry

Topic 8: rn | crying rn | crying | cant | im | crypto | cry

Topic 9: crying | rn | im | crying rn | cant | cry | crypto

Topic 10: rn | crypto | cant | crying | cry | im | crying rn



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Bad - Topic Models

In [17]:
n_top_words = 10
topic_word, vocab = generate_topic_models("bad")
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print ("Topic " + str(i+1) + ": " + " | ".join(topic_words) + "\n")

Topic 1: bad | im | bad rn | rn

Topic 2: bad | rn | bad rn | im

Topic 3: rn | bad | im | bad rn

Topic 4: bad | rn | bad rn | im

Topic 5: im | bad rn | bad | rn

Topic 6: bad | rn | bad rn | im

Topic 7: im | bad | rn | bad rn

Topic 8: bad rn | im | rn | bad

Topic 9: rn | bad rn | im | bad

Topic 10: rn | bad rn | bad | im



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Thank you: https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

In [18]:
#Reminder:
df

,Unnamed: 0,Unnamed: 0.1,Tweet Date,Tweet ID,Tweet Text,tweet_source,tweet_retweet_count,tweet_favorite_count,word_count,char_count,...,month,day,unique_word_count,stop_word_count,url_count,mean_word_length,hashtag_count,mention_count,burnout,burnout_indicators
0,0,0,2021-01-19,1351590096752017416,I'm a big music person. It speaks to me in all...,Twitter for Android,0,6,27,142,...,1,19,25,15,1,4.296296,0,0,1,mad
1,1,1,2021-01-19,1351555038863368193,@JohnWHarris15 I adore you and your unlimited ...,Twitter for Android,0,1,9,59,...,1,19,9,4,0,5.555556,0,1,1,
2,2,2,2021-01-19,1351554892926750720,@bubblydncer I have replied to countless texts...,Twitter for Android,0,1,22,142,...,1,19,20,10,1,5.500000,0,1,1,
3,3,3,2021-01-19,1351554395234824192,@MelBeer93 Ahhh I love this!,Twitter for Android,0,0,5,29,...,1,19,5,1,0,4.800000,0,1,0,
4,4,4,2021-01-19,1351554266247426050,@TheKimClub https://t.co/UUx3417HcD,Twitter for Android,0,1,2,35,...,1,19,2,0,1,17.000000,0,1,1,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1874,1874,2026,2021-01-11,1348687433978798080,in my lecture rn talking about covid,Twitter for iPhone,0,0,7,37,...,1,11,7,3,0,4.285714,0,0,1,
1875,1875,2027,2021-01-11,1348687432938647552,@buzzycalz @skinnyqweeen @hearteyezshawty im i...,Twitter for iPhone,0,2,27,174,...,1,11,26,11,0,5.444444,0,3,1,
1876,1876,2028,2021-01-11,1348687432913473536,"@Death13__ Also this is just streams, videos a...",Twitter for Android,0,1,16,84,...,1,11,16,8,0,4.312500,0,1,1,
1877,1877,2029,2021-01-11,1348687432904892416,@Lyssss69 Honestly I just saw these on tiktok ...,Twitter for iPhone,0,1,15,74,...,1,11,14,8,0,4.000000,0,1,0,


In [19]:
#Split data into training, validation and test sets
y=df['burnout']

xtrain, xtest, ytrain, ytest = train_test_split(df['Tweet Text'].values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [20]:
print (xtrain.shape)
print (xtest.shape)

(1503,)
(376,)


## Baseline Models

### TF-IDF

In [21]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to training set (supervised learning)
tfv.fit(list(xtrain))
xtrain_tfv =  tfv.transform(xtrain) 
xtest_tfv =  tfv.transform(xtest) 

### Logistic Regression

In [22]:
# Fitting a simple Logistic Regression on TFIDF

clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xtest_tfv)


print(classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.79      0.41      0.54       133
           1       0.74      0.94      0.83       243

    accuracy                           0.75       376
   macro avg       0.77      0.67      0.68       376
weighted avg       0.76      0.75      0.73       376



* Recall: how many of this class you find over the whole # of elements of this class

* Precision: how many are correctly classified among that class

* f1-score: harmonic mean between Pr and Recall

* support: how many of each class. (There is class imbalance since 1-burnout is close to double the amount of 0-non burnout).

* accuracy: percentage of correct predictions

* macro avg: 𝑠𝑐𝑜𝑟𝑒𝑚𝑎𝑐𝑟𝑜-𝑎𝑣𝑔=0.5⋅𝑠𝑐𝑜𝑟𝑒𝑐𝑙𝑎𝑠𝑠 0+0.5⋅𝑠𝑐𝑜𝑟𝑒𝑐𝑙𝑎𝑠𝑠 1

* weighted avg: 𝑠𝑐𝑜𝑟𝑒𝑤𝑒𝑖𝑔ℎ𝑡𝑒𝑑-𝑎𝑣𝑔=0.998⋅𝑠𝑐𝑜𝑟𝑒𝑐𝑙𝑎𝑠𝑠 0+0.002⋅𝑠𝑐𝑜𝑟𝑒𝑐𝑙𝑎𝑠𝑠 1

Conclusion: 
* Model did a better job at correctly classifying burnout due to higher f1-score. 

* However, non-burnout class has greater Precision.

* Given imbalanced dataset, not good to look at accuracy since it can just give illusion of good model performance. This can happen when class dominates another class such as in this case. And model does good job predicting popular class but does not do good job predicting non-popular class. 

* macro avg: f1-score gets penalized when model does not classify minority class well (you want this to be high). In this case, there was high precision/accuracy, but low recall/coverage. So the macro avg for f1-score is not too low and not too high.

* weighted avg: if the f1-score for this is high then the model is favoring the majority class, which is not what we want (you want this to be low). In this case, given the class imbalance, this is what is happening with our model.

Let's take a look at the confusion matrix which can give us a clearer picture of what is expected and what model ended up predicting.

In [23]:
confusion_matrix(ytest, predictions)

array([[ 54,  79],
       [ 14, 229]])

Along the vertical axis is the actual classes  0 as the top left and 1 as the bottom left. And along the horizontal axis 0 is the top left and 1 as the top right.

From the output, we can see that the model did a very good job predicting burnout tweets and a mediocre job with non-burnout tweets. There are far greater false positives than false negatives which is expected given that class imbalance is heavily skewed towards positive class. 

Therefore, can look at weighing burnout class less when model trains on data. Another alternative is to use a different vectorizer- CountVectorizer to represent text for training.

### CountVectorizer

In [24]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')



# Fitting Count Vectorizer to training set
xtrain_ctv =  ctv.fit_transform(xtrain) 
xtest_ctv = ctv.transform(xtest)

In [25]:
# Fitting a simple Logistic Regression on Counts

clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict(xtest_ctv)


print(classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.87      0.36      0.51       133
           1       0.74      0.97      0.84       243

    accuracy                           0.76       376
   macro avg       0.80      0.67      0.67       376
weighted avg       0.78      0.76      0.72       376



Count Vectorizer performed better since minority class has higher Precision though Recall was lower than seen in TF-IDF results.

Let's take a look at the confusion matrix to see more.

In [26]:
confusion_matrix(ytest, predictions)

array([[ 48,  85],
       [  7, 236]])

We can see with Count Vectorizer there are more false positives and less false negatives. In our situation we want less false negatives because we do not want to miss anyone who is burntout. The cost is higher for someone who is not diagnosed versus someone who is misdiagnosed as having it when that is not the case.

So far CountVectorizer with Log Reg is in the lead with best model performance.

Now let us give Naive Bayes a chance.

### Naive Bayes

In [27]:
#On TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xtest_tfv)

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.81      0.41      0.54       133
           1       0.74      0.95      0.83       243

    accuracy                           0.76       376
   macro avg       0.78      0.68      0.69       376
weighted avg       0.77      0.76      0.73       376



In [28]:
print (confusion_matrix(ytest, predictions))

[[ 54  79]
 [ 13 230]]


Not many drastic changes with classification report output. And with confusion matrix, there are more false negatives. So let us try NB on Counts.  

In [29]:
#On Counts Vectorizer
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict(xtest_ctv)

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.70      0.43      0.53       133
           1       0.74      0.90      0.81       243

    accuracy                           0.73       376
   macro avg       0.72      0.66      0.67       376
weighted avg       0.73      0.73      0.71       376



In [30]:
confusion_matrix(ytest, predictions)

array([[ 57,  76],
       [ 24, 219]])

This is a huge change in model performance and not an improvement. Though recall improved for minority class, there is a huge spike in false negatives. 

So Logistic Regression using Counts is still the top ranked. Let us try another old algorithm called SVM (Support Vector Machines).

### Support Vector Machine

Due to how long the algorithm takes to predict, will need to reduce TFIDF vector as well as standardize the data for SVM to work.

In [31]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xtest_svd = svd.transform(xtest_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xtest_svd_scl = scl.transform(xtest_svd)

In [32]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict(xtest_svd_scl)

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.79      0.37      0.50       133
           1       0.73      0.95      0.83       243

    accuracy                           0.74       376
   macro avg       0.76      0.66      0.66       376
weighted avg       0.75      0.74      0.71       376



In [33]:
confusion_matrix(ytest, predictions)

array([[ 49,  84],
       [ 13, 230]])

Given minority class' Precision and confusion matrix, similar performance to top-ranked Log Reg using Count Vectorizer. Let's see if we can do even better with non-baseline models.

## Non-Baseline Model

### XGBoost

This is the most popular ML algorithm for competition winners. So let us see how it performs. 

In [34]:
# Fitting XGBoost on TFIDF
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict(xtest_tfv.tocsc())

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.80      0.54      0.65       133
           1       0.79      0.93      0.85       243

    accuracy                           0.79       376
   macro avg       0.79      0.73      0.75       376
weighted avg       0.79      0.79      0.78       376



In [35]:
confusion_matrix(ytest, predictions)

array([[ 72,  61],
       [ 18, 225]])

Wow! What a game-changer. I can see why XGBoost is competition winners' go-to for winning. Significant changes to model performance happened in several areas:

* 0/non-burnout class had higher Recall/coverage than all prior models 

* f1-score for minority and majority classes have highest score versus prior models

* macro avg is higher than all prior models

* overall incorrect predictions is the least amount out of all models though its false negatives could be lower

Refined metric of success is:
1. low # false predictions (low false pos' and low false neg's) - this has higher wt than low false neg's
1. Low false negatives

Given the above, XGBoost on TFIDF is the current winner!

In [36]:
# Fitting XGBoost on Counts
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict(xtest_ctv.tocsc())

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.79      0.54      0.64       133
           1       0.79      0.92      0.85       243

    accuracy                           0.79       376
   macro avg       0.79      0.73      0.75       376
weighted avg       0.79      0.79      0.78       376



In [37]:
confusion_matrix(ytest, predictions)

array([[ 72,  61],
       [ 19, 224]])

XGBoost on Count Vectorizer performed similar, but performed slightly worse when it came to minority class Precision, majority class Recall, minority class f1-score and incorrect classifications (false pos, false neg).

Given that XGBoost performed so well, let us try and see how it performs using svd features with some slight variation in XGBoost parameters.

In [38]:
# Fitting XGBoost on Support Vector dimensions/features (svd) using prior model parameters
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict(xtest_svd)

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.73      0.47      0.57       133
           1       0.76      0.91      0.82       243

    accuracy                           0.75       376
   macro avg       0.74      0.69      0.70       376
weighted avg       0.75      0.75      0.73       376



In [39]:
confusion_matrix(ytest, predictions)

array([[ 62,  71],
       [ 23, 220]])

In [40]:
# Fitting XGBoost on svd using 1 model parameter
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict(xtest_svd)

print (classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.65      0.42      0.51       133
           1       0.73      0.88      0.80       243

    accuracy                           0.72       376
   macro avg       0.69      0.65      0.66       376
weighted avg       0.71      0.72      0.70       376



In [41]:
confusion_matrix(ytest, predictions)

array([[ 56,  77],
       [ 30, 213]])

Both XGBoost on different parameters did not perform better than XGBoost on TF-IDF.